In [6]:
from neo4j import GraphDatabase
from pyproj import CRS, Transformer
from shapely.ops import nearest_points
from shapely.geometry import Point

# heuristic for score

# Closest existing charging location + density + avg home value + number of vehicles + population density

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")
driver = GraphDatabase.driver(URI, auth=AUTH)


# create a score for the candidate locations
# based on the closest existing charging location + density + avg home value + number of vehicles + population density
def calculate_score(candidate, pc4, municipality):

    print(candidate)
    closest_charging_location_distance = candidate["distance_to_nearest"]
    print(f"Closest charging location distance: {closest_charging_location_distance}")
    ev_charger_density = pc4["density"]
    print(f"EV charger density: {ev_charger_density}")
    avg_home_value = municipality["home_value"]
    print(f"Average home value: {avg_home_value}")
    number_of_vehicles = municipality["vehicles"]
    print(f"Number of vehicles: {number_of_vehicles}")
    population_density = municipality["population_density"]
    print(f"Population density: {population_density}")
    score = (closest_charging_location_distance * 0.4 +
                ev_charger_density * 0.2 +
                avg_home_value * 0.2 +
                number_of_vehicles * 0.1 +
                population_density * 0.1)
    return score

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
    

        # get the pc4 that are in the municipality
        municipality = "Krimpenerwaard"
        query = """
        MATCH (p:PC4Area) - [:IS_LOCATED_IN] -> (m:Municipality {name: $municipality})
        RETURN p
        """
        result = session.run(query, municipality=municipality)
        pc4_areas = [record["p"] for record in result]
        if not pc4_areas:
            print(f"No PC4 areas found in the municipality {municipality}.")
            exit()
        print(f"Found {len(pc4_areas)} PC4 areas in the municipality {municipality}.")

        # get the candidate locations in the pc4 areas individually
        # and return the candidate locations in the municipality
        # if no pc4 areas are found, exit
        candidate_locations = []
        for pc4 in pc4_areas:
            query = """
            MATCH (c:CandidateLocation) - [:IS_LOCATED_IN] -> (p:PC4Area {pc4_code: $pc4_code})
            WITH c, p
            MATCH (c) - [:IS_LOCATED_IN] -> (m:Municipality {name: $municipality})
            RETURN properties(c) AS c, properties(p) AS p, properties(m) AS m
            """
            result = session.run(query, pc4_code=pc4["pc4_code"], municipality=municipality)
            # for record in result:
            #     print("candidate location:", record["c"])
            #     print("pc4 area:", record["p"])
            #     print("municipality:", record["m"])
            candidates = [(record["c"], record["p"], record["m"]) for record in result]
            if not candidates:
                print(f"No candidates found for the PC4 code {pc4['pc4_code']}.")
                continue
            for candidate in candidate_locations:
                if candidate[0] is None or candidate[1] is None or candidate[2] is None:
                    print(f"Candidate {candidate} has missing data, skipping.")
                    continue
                # calculate the score for each candidate location
                score = calculate_score(candidate[0], candidate[1], candidate[2])
                candidate[0]["score"] = score
                # add the score to the candidate location and update neo4j
                update_query = """
                MATCH (c:CandidateLocation {lon: $lon, lat: $lat})
                SET c.score = $score
                RETURN c
                """
                session.run(update_query, lon=candidate[0]["lon"], lat = candidate[0]["lat"], score=score)
            print(f"Found {len(candidates)} candidates for PC4 code {pc4['pc4_code']}.")
            candidate_locations.extend(candidates)
        if not candidate_locations:
            exit()

        
        


    driver.close()


Found 12 PC4 areas in the municipality Krimpenerwaard.
Found 1 candidates for PC4 code 2872.
{'lon': 4.871950344247892, 'distance_to_nearest': 521.8674187538921, 'nearest_lon': 4.867512, 'lat': 51.946562561145534, 'nearest_lat': 51.947493}
Closest charging location distance: 521.8674187538921
EV charger density: 0.0
Average home value: 404000
Number of vehicles: 46780
Population density: 3.583162854131796
Found 11 candidates for PC4 code 2821.
{'lon': 4.871950344247892, 'distance_to_nearest': 521.8674187538921, 'nearest_lon': 4.867512, 'lat': 51.946562561145534, 'nearest_lat': 51.947493, 'score': 85687.10528378698}
Closest charging location distance: 521.8674187538921
EV charger density: 0.0
Average home value: 404000
Number of vehicles: 46780
Population density: 3.583162854131796
{'lon': 4.7699034, 'distance_to_nearest': 5195.2167857709255, 'nearest_lon': 4.7758414, 'lat': 51.9712063, 'nearest_lat': 51.9997146}
Closest charging location distance: 5195.2167857709255
EV charger density: